In [ ]:
from requests_html import HTMLSession
import time
import pymysql
import random

import mysql.connector
import urllib.request
import   sqlhelper
import   utils
import re
import os



def getTitleImg(url,id):
    path = 'E:\\phpstudy_pro\\WWW\\images\\' +str(id)+'\\' 
   # 将图片保存到D:\\test文件夹中，如果没有test文件夹则创建
    if not os.path.isdir(path):  
        os.makedirs(path)  
    paths = path+'\\'      #保存在test路径下 
    x="imgtitle"+str(id)
    imgurl="http:"+url
    ext=(imgurl.rpartition("."))[2]
    print(ext) 
    print("titleimgurl=",imgurl)
    urllib.request.urlretrieve(imgurl,'{0}{1}.{2}'.format(paths,x,ext))
    titleimgurl="/images/"+str(id)+"/imgtitle"+str(id)+".jpg"  
    print("测试：titleimgurl：---",titleimgurl)
    return  titleimgurl



sql="SELECT * FROM urls WHERE  date2 IS NULL"
queryresult=sqlhelper.select_all(sql,[])

#print(len(queryresult))


for x in queryresult:
  try:
    categoryid=x[0]
    print(categoryid)
    category_id=int(x[0])
    url_t=x[1]
    print(x[1])   
  #构造地址链接  
 # 

  
    for i in range(1,100,2):
        id=0  #初始值
        try:
         #  
           url=url_t+"&page="+str(i)+"&s=11&click=0"
           print(url)
           second=random.randint(20,100)
           print("抓取页面间隔{0}秒".format(second))
           time.sleep(second)
            
           
      #label .begin
           session = HTMLSession()
     # try:
           r = session.get(url)
           url=url_t
        
           list_a=r.html.find('div#J_goodsList')

           path="//li"
#print(list_a[0].html)
           content=list_a[0].html
           path2="//@data-lazy-img"
          # path3="//em/text()"
          # path3="//div[@class='p-name']/a/@title"
           path3="//div/a/em/text()"  
           path4="//i/text()"
           path5="//a/text()"
           path6="//span[@class='J_im_icon']/a/text()"
           path7="//a[@class='curr-shop hd-shopname']/text()"
           path8="//a/img[data-lazy-img]"
           path9="//div[@class='p-img']/a/@href"
           path10="//href"
           id=0
#  content="<html><body style='text-align:center;'><table  border='1' style='border-collapse:collapse;margin:0px auto;border:1px solid ;'><tr style='background-color:red;'><th>选择框</th><th>ID</th><th>产品图</th><th>产品名</th><th>价格</th><th>店铺名</th><th>生成时间</th><th>操作</th></tr>"
           valist=[]
         #  time.sleep(random.randint(15,110))
           for li in (r.html.find('div#J_goodsList'))[0].xpath(path):

                   img_url=li.xpath(path2)

                   name=li.xpath(path3)[0]
                   name=name.replace('\r','').replace('\n','').replace('\t','')
                   print(name)
    #   

     #  print(li.xpath(path4)[0])
    #   print(li.xpath(path5)[0])    
                   price=li.xpath(path4)[0]
   #    print(li.xpath(path6))
                   store_name=li.xpath(path7)
                   s_name=""
                   if store_name==[]:
                        s_name=""
                   else:
                        s_name=store_name[0]
            
      # print(s_name)
                   product_url=li.xpath(path9)
                   if "item.x.com"  in product_url[0]:
       
                       sql="select * from spider where url=%s"   #查询地址是否已经添加到数据库
                       print("product_url[0]:",product_url[0])
                       result=sqlhelper.select_one(sql, product_url[0]) 
                       print("result:",result)
                       if   result == None:
                            # titleimgurl=getTitleImg(img_url[0],id)
                             tup1 = [name,img_url[0],price,s_name,time.strftime('%Y-%m-%d %H:%M:%S',time.localtime()),category_id,product_url[0]]    
                  # valist.append(tup1)
                           
                             sql = "INSERT INTO spider(title, img, price, store,date,categoryid,url) VALUES (%s, %s,%s, %s, %s, %s,%s )"
                            # print(res, "记录写入成功。")
                             res = sqlhelper.insert_one(sql, tup1)
                             print(res, "记录写入成功。")                                
                                
                             sql="select * from spider where img=%s"
                             resultimg=sqlhelper.select_one(sql, img_url[0]) 
                             print("id：",resultimg[0])
                             print("标题图地址：",resultimg[2])
                             id=resultimg[0]
                             newtitleimgurl=getTitleImg(img_url[0],id)
                             

                             print("newtitleimgurl:",newtitleimgurl)

                             sql = "UPDATE spider SET img=%s WHERE id= %s"
                       
                             booleantag=sqlhelper.update_one(sql,[newtitleimgurl,str(id)])
                             print("booleantag=",booleantag)

           #time1=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())
           
          # mycursor.executemany(sql, tuple(valist))
        except:
                delpath = 'f:\\phpstudy_pro\\WWW\\images\\' +str(id)+'\\' 
                remove_dir(delpath,print_result=True)
                time.sleep(random.randint(15,110))
                continue
   # sql = "INSERT INTO spider(title, img, price, store,date,categoryid,url) VALUES (%s, %s,%s, %s, %s, %s,%s )"
    sql = "UPDATE urls SET tag = %s,date2=%s  WHERE id= %s"
    res = sqlhelper.update_one(sql, ["Y",time.strftime('%Y-%m-%d %H:%M:%S',time.localtime()),str(category_id)])
    print(res, "一个产品分类地址记录写完。") 
  except:
                second=random.randint(1500,250)
                print("抓取异常休息{}秒再继续，时间点：{2}".format(second,time1=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())))
                time.sleep(second)
                continue
        
        